<a href="https://colab.research.google.com/github/dangkhoa152/3122410187_TranLeDangKhoa_DataAnalysis/blob/main/Chapter10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHAPTER 10

In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [2]:
import numpy as np
import pandas as pd

## How to Think About Group Operations

In [3]:
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1],
                                      dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})
df

,key1,key2,data1,data2
0,a,1,-0.204708,0.281746
1,a,2,0.478943,0.769023
2,None,1,-0.519439,1.246435
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221
5,a,<NA>,1.393406,0.274992
6,None,1,0.092908,0.228913


In [4]:
grouped = df["data1"].groupby(df["key1"])
grouped

In [5]:
grouped.mean()

,data1
key1,
a,0.555881
b,0.705025


In [6]:
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()
means

key1  key2
a     1      -0.204708
      2       0.478943
b     1       1.965781
      2      -0.555730
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,1,2
key1,,
a,-0.204708,0.478943
b,1.965781,-0.555730


In [8]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]
df["data1"].groupby([states, years]).mean()

CA  2005    0.936175
    2006   -0.519439
OH  2005   -0.380219
    2006    1.029344
Name: data1, dtype: float64

In [9]:
df.groupby("key1").mean()
df.groupby("key2").mean(numeric_only=True)
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1    -0.204708  0.281746
     2     0.478943  0.769023
b    1     1.965781 -1.296221
     2    -0.555730  1.007189

In [10]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [11]:
df.groupby("key1", dropna=False).size()
df.groupby(["key1", "key2"], dropna=False).size()

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [12]:
df.groupby("key1").count()

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


## Iterating over Groups

In [13]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1 -0.204708  0.281746
1    a     2  0.478943  0.769023
5    a  <NA>  1.393406  0.274992
b
  key1  key2     data1     data2
3    b     2 -0.555730  1.007189
4    b     1  1.965781 -1.296221


In [14]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', np.int64(1))
  key1  key2     data1     data2
0    a     1 -0.204708  0.281746
('a', np.int64(2))
  key1  key2     data1     data2
1    a     2  0.478943  0.769023
('b', np.int64(1))
  key1  key2     data1     data2
4    b     1  1.965781 -1.296221
('b', np.int64(2))
  key1  key2    data1     data2
3    b     2 -0.55573  1.007189


In [15]:
pieces = {name: group for name, group in df.groupby("key1")}
pieces["b"]

,key1,key2,data1,data2
3,b,2,-0.555730,1.007189
4,b,1,1.965781,-1.296221


In [16]:
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns")

/tmp/ipython-input-1884927282.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  grouped = df.groupby({"key1": "key", "key2": "key",


In [17]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)

data
      data1     data2
0 -0.204708  0.281746
1  0.478943  0.769023
2 -0.519439  1.246435
3 -0.555730  1.007189
4  1.965781 -1.296221
5  1.393406  0.274992
6  0.092908  0.228913
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


## Selecting a Column or Subset of Columns

In [18]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1     0.281746
     2     0.769023
b    1    -1.296221
     2     1.007189

In [19]:
s_grouped = df.groupby(["key1", "key2"])["data2"]
s_grouped
s_grouped.mean()

key1  key2
a     1       0.281746
      2       0.769023
b     1      -1.296221
      2       1.007189
Name: data2, dtype: float64

## Grouping with Dictionaries and Series

In [20]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,1.352917,0.886429,-2.001637,-0.371843,1.669025
Steve,-0.438570,-0.539741,0.476985,3.248944,-1.021228
Wanda,-0.577087,NaN,NaN,0.523772,0.000940
Jill,1.343810,-0.713544,-0.831154,-2.370232,-1.860761
Trey,-0.860757,0.560145,-1.265934,0.119827,-1.063512


In [21]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}

In [22]:
by_column = people.groupby(mapping, axis="columns")
by_column.sum()

/tmp/ipython-input-2628240624.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  by_column = people.groupby(mapping, axis="columns")


,blue,red
Joe,-2.373480,3.908371
Steve,3.725929,-1.999539
Wanda,0.523772,-0.576147
Jill,-3.201385,-1.230495
Trey,-1.146107,-1.364125


In [23]:
map_series = pd.Series(mapping)
map_series
people.groupby(map_series, axis="columns").count()

/tmp/ipython-input-1085982663.py:3: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  people.groupby(map_series, axis="columns").count()


,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


## Grouping with Functions

In [24]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.352917,0.886429,-2.001637,-0.371843,1.669025
4,0.483052,-0.153399,-2.097088,-2.250405,-2.924273
5,-1.015657,-0.539741,0.476985,3.772716,-1.020287


In [25]:
key_list = ["one", "one", "one", "two", "two"]
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,1.352917,0.886429,-2.001637,-0.371843,1.669025
4,two,-0.860757,-0.713544,-1.265934,-2.370232,-1.860761
5,one,-0.577087,-0.539741,0.476985,0.523772,-1.021228


## Grouping by Index Levels

In [26]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                    [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      0.332883 -2.359419 -0.199543 -1.541996 -0.970736
1     -1.307030  0.286350  0.377984 -0.753887  0.331286
2      1.349742  0.069877  0.246674 -0.011862  1.004812
3      1.327195 -0.919262 -1.549106  0.022185  0.758363

In [27]:
hier_df.groupby(level="cty", axis="columns").count()

/tmp/ipython-input-2795167867.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  hier_df.groupby(level="cty", axis="columns").count()


cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## Data Aggregation

In [28]:
df
grouped = df.groupby("key1")
grouped["data1"].nsmallest(2)

key1   
a     0   -0.204708
      1    0.478943
b     3   -0.555730
      4    1.965781
Name: data1, dtype: float64

In [29]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,key2,data1,data2
key1,,,
a,1,1.598113,0.494031
b,1,2.521511,2.303410


In [30]:
grouped.describe()

key2                                           data1            ...  \
     count mean       std  min   25%  50%   75%  max count      mean  ...   
key1                                                                  ...   
a      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   3.0  0.555881  ...   
b      2.0  1.5  0.707107  1.0  1.25  1.5  1.75  2.0   2.0  0.705025  ...   

                         data2                                          \
           75%       max count      mean       std       min       25%   
key1                                                                     
a     0.936175  1.393406   3.0  0.441920  0.283299  0.274992  0.278369   
b     1.335403  1.965781   2.0 -0.144516  1.628757 -1.296221 -0.720368   

                                    
           50%       75%       max  
key1                                
a     0.281746  0.525384  0.769023  
b    -0.144516  0.431337  1.007189  

[2 rows x 24 columns]

## Column-Wise and Multiple Function Application

In [32]:
import seaborn as sns
import pandas as pd

# Load trực tiếp dataset tips từ seaborn
tips = sns.load_dataset("tips")

# Xem 5 dòng đầu
print(tips.head())


   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


In [33]:
tips["tip_pct"] = tips["tip"] / tips["total_bill"]
tips.head()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808


In [34]:
grouped = tips.groupby(["day", "smoker"])

/tmp/ipython-input-3355053125.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = tips.groupby(["day", "smoker"])


In [35]:
grouped_pct = grouped["tip_pct"]
grouped_pct.agg("mean")

day   smoker
Thur  Yes       0.163863
      No        0.160298
Fri   Yes       0.174783
      No        0.151650
Sat   Yes       0.147906
      No        0.158048
Sun   Yes       0.187250
      No        0.160113
Name: tip_pct, dtype: float64

In [36]:
grouped_pct.agg(["mean", "std", peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Thur Yes     0.163863  0.039389      0.151240
     No      0.160298  0.038774      0.193350
Fri  Yes     0.174783  0.051293      0.159925
     No      0.151650  0.028123      0.067349
Sat  Yes     0.147906  0.061375      0.290095
     No      0.158048  0.039767      0.235193
Sun  Yes     0.187250  0.154134      0.644685
     No      0.160113  0.042347      0.193226

In [37]:
grouped_pct.agg([("average", "mean"), ("stdev", np.std)])

/tmp/ipython-input-1734782830.py:1: FutureWarning: The provided callable <function std at 0x7d21a9f70ae0> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  grouped_pct.agg([("average", "mean"), ("stdev", np.std)])


average     stdev
day  smoker                    
Thur Yes     0.163863  0.039389
     No      0.160298  0.038774
Fri  Yes     0.174783  0.051293
     No      0.151650  0.028123
Sat  Yes     0.147906  0.061375
     No      0.158048  0.039767
Sun  Yes     0.187250  0.154134
     No      0.160113  0.042347

In [38]:
functions = ["count", "mean", "max"]
result = grouped[["tip_pct", "total_bill"]].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Thur Yes         17  0.163863  0.241255         17  19.190588  43.11
     No          45  0.160298  0.266312         45  17.113111  41.19
Fri  Yes         15  0.174783  0.263480         15  16.813333  40.17
     No           4  0.151650  0.187735          4  18.420000  22.75
Sat  Yes         42  0.147906  0.325733         42  21.276667  50.81
     No          45  0.158048  0.291990         45  19.661778  48.33
Sun  Yes         19  0.187250  0.710345         19  24.120000  45.35
     No          57  0.160113  0.252672         57  20.506667  48.17

In [39]:
result["tip_pct"]

count      mean       max
day  smoker                           
Thur Yes        17  0.163863  0.241255
     No         45  0.160298  0.266312
Fri  Yes        15  0.174783  0.263480
     No          4  0.151650  0.187735
Sat  Yes        42  0.147906  0.325733
     No         45  0.158048  0.291990
Sun  Yes        19  0.187250  0.710345
     No         57  0.160113  0.252672

In [40]:
ftuples = [("Average", "mean"), ("Variance", np.var)]
grouped[["tip_pct", "total_bill"]].agg(ftuples)

/tmp/ipython-input-4198102554.py:2: FutureWarning: The provided callable <function var at 0x7d21a9f70c20> is currently using SeriesGroupBy.var. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "var" instead.
  grouped[["tip_pct", "total_bill"]].agg(ftuples)


tip_pct           total_bill            
              Average  Variance    Average    Variance
day  smoker                                           
Thur Yes     0.163863  0.001551  19.190588   69.808518
     No      0.160298  0.001503  17.113111   59.625081
Fri  Yes     0.174783  0.002631  16.813333   82.562438
     No      0.151650  0.000791  18.420000   25.596333
Sat  Yes     0.147906  0.003767  21.276667  101.387535
     No      0.158048  0.001581  19.661778   79.908965
Sun  Yes     0.187250  0.023757  24.120000  109.046044
     No      0.160113  0.001793  20.506667   66.099980

In [41]:
grouped.agg({"tip" : np.max, "size" : "sum"})
grouped.agg({"tip_pct" : ["min", "max", "mean", "std"],
             "size" : "sum"})

/tmp/ipython-input-3437405047.py:1: FutureWarning: The provided callable <function max at 0x7d21a9f4ff60> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  grouped.agg({"tip" : np.max, "size" : "sum"})


tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Thur Yes     0.090014  0.241255  0.163863  0.039389   40
     No      0.072961  0.266312  0.160298  0.038774  112
Fri  Yes     0.103555  0.263480  0.174783  0.051293   31
     No      0.120385  0.187735  0.151650  0.028123    9
Sat  Yes     0.035638  0.325733  0.147906  0.061375  104
     No      0.056797  0.291990  0.158048  0.039767  115
Sun  Yes     0.065660  0.710345  0.187250  0.154134   49
     No      0.059447  0.252672  0.160113  0.042347  167

## Returning Aggregated Data Without Row Indexes

In [42]:
grouped = tips.groupby(["day", "smoker"], as_index=False)
grouped.mean(numeric_only=True)

/tmp/ipython-input-494164758.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = tips.groupby(["day", "smoker"], as_index=False)


,day,smoker,total_bill,tip,size,tip_pct
0,Thur,Yes,19.190588,3.030000,2.352941,0.163863
1,Thur,No,17.113111,2.673778,2.488889,0.160298
2,Fri,Yes,16.813333,2.714000,2.066667,0.174783
3,Fri,No,18.420000,2.812500,2.250000,0.151650
4,Sat,Yes,21.276667,2.875476,2.476190,0.147906
5,Sat,No,19.661778,3.102889,2.555556,0.158048
6,Sun,Yes,24.120000,3.516842,2.578947,0.187250
7,Sun,No,20.506667,3.167895,2.929825,0.160113


## Apply: General split-apply-combine

In [43]:
def top(df, n=5, column="tip_pct"):
    return df.sort_values(column, ascending=False)[:n]
top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525


In [44]:
tips.groupby("smoker").apply(top)

/tmp/ipython-input-2530541573.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips.groupby("smoker").apply(top)
/tmp/ipython-input-2530541573.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker").apply(top)


total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
Yes    172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
No     232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746

In [45]:
tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")

/tmp/ipython-input-643835440.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")
/tmp/ipython-input-643835440.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby(["smoker", "day"]).apply(top, n=1, column="total_bill")


total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
Yes    Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   
       Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
No     Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
       Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   

                  tip_pct  
smoker day                 
Yes    Thur 197  0.115982  
       Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
No     Thur 142  0.121389  
       Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799

In [46]:
result = tips.groupby("smoker")["tip_pct"].describe()
result
result.unstack("smoker")

/tmp/ipython-input-2216311278.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  result = tips.groupby("smoker")["tip_pct"].describe()


smoker
count  Yes        93.000000
       No        151.000000
mean   Yes         0.163196
       No          0.159328
std    Yes         0.085119
       No          0.039910
min    Yes         0.035638
       No          0.056797
25%    Yes         0.106771
       No          0.136906
50%    Yes         0.153846
       No          0.155625
75%    Yes         0.195059
       No          0.185014
max    Yes         0.710345
       No          0.291990
dtype: float64

## Suppressing the Group Keys

In [47]:
tips.groupby("smoker", group_keys=False).apply(top)

/tmp/ipython-input-3851957478.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  tips.groupby("smoker", group_keys=False).apply(top)
/tmp/ipython-input-3851957478.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tips.groupby("smoker", group_keys=False).apply(top)


,total_bill,tip,sex,smoker,day,time,size,tip_pct
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746


## Quantile and Bucket Analysis

In [48]:
frame = pd.DataFrame({"data1": np.random.standard_normal(1000),
                      "data2": np.random.standard_normal(1000)})
frame.head()
quartiles = pd.cut(frame["data1"], 4)
quartiles.head(10)

,data1
0,"(-1.23, 0.489]"
1,"(0.489, 2.208]"
2,"(-1.23, 0.489]"
3,"(-1.23, 0.489]"
4,"(0.489, 2.208]"
5,"(0.489, 2.208]"
6,"(-1.23, 0.489]"
7,"(-1.23, 0.489]"
8,"(-2.956, -1.23]"
9,"(-1.23, 0.489]"


In [49]:
def get_stats(group):
    return pd.DataFrame(
        {"min": group.min(), "max": group.max(),
        "count": group.count(), "mean": group.mean()}
    )

grouped = frame.groupby(quartiles)
grouped.apply(get_stats)

/tmp/ipython-input-2015905501.py:7: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = frame.groupby(quartiles)


min       max  count      mean
data1                                                     
(-2.956, -1.23] data1 -2.949343 -1.230179     94 -1.658818
                data2 -3.399312  1.670835     94 -0.033333
(-1.23, 0.489]  data1 -1.228918  0.488675    598 -0.329524
                data2 -2.989741  3.260383    598 -0.002622
(0.489, 2.208]  data1  0.489965  2.200997    298  1.065727
                data2 -3.745356  2.954439    298  0.078249
(2.208, 3.928]  data1  2.212303  3.927528     10  2.644253
                data2 -1.929776  1.765640     10  0.024750

In [50]:
grouped.agg(["min", "max", "count", "mean"])

data1                               data2                  \
                      min       max count      mean       min       max count   
data1                                                                           
(-2.956, -1.23] -2.949343 -1.230179    94 -1.658818 -3.399312  1.670835    94   
(-1.23, 0.489]  -1.228918  0.488675   598 -0.329524 -2.989741  3.260383   598   
(0.489, 2.208]   0.489965  2.200997   298  1.065727 -3.745356  2.954439   298   
(2.208, 3.928]   2.212303  3.927528    10  2.644253 -1.929776  1.765640    10   

                           
                     mean  
data1                      
(-2.956, -1.23] -0.033333  
(-1.23, 0.489]  -0.002622  
(0.489, 2.208]   0.078249  
(2.208, 3.928]   0.024750

In [51]:
quartiles_samp = pd.qcut(frame["data1"], 4, labels=False)
quartiles_samp.head()
grouped = frame.groupby(quartiles_samp)
grouped.apply(get_stats)

min       max  count      mean
data1                                           
0     data1 -2.949343 -0.685484    250 -1.212173
      data2 -3.399312  2.628441    250 -0.027045
1     data1 -0.683066 -0.030280    250 -0.368334
      data2 -2.630247  3.260383    250 -0.027845
2     data1 -0.027734  0.618965    250  0.295812
      data2 -3.056990  2.458842    250  0.014450
3     data1  0.623587  3.927528    250  1.248875
      data2 -3.745356  2.954439    250  0.115899

## Example: Filling Missing Values with Group-Specific Values

In [52]:
s = pd.Series(np.random.standard_normal(6))
s[::2] = np.nan
s
s.fillna(s.mean())

,0
0,-0.767366
1,0.227290
2,-0.767366
3,-2.153545
4,-0.767366
5,-0.375842


In [53]:
states = ["Ohio", "New York", "Vermont", "Florida",
          "Oregon", "Nevada", "California", "Idaho"]
group_key = ["East", "East", "East", "East",
             "West", "West", "West", "West"]
data = pd.Series(np.random.standard_normal(8), index=states)
data

,0
Ohio,0.329939
New York,0.981994
Vermont,1.105913
Florida,-1.613716
Oregon,1.561587
Nevada,0.406510
California,0.359244
Idaho,-0.614436


In [54]:
data[["Vermont", "Nevada", "Idaho"]] = np.nan
data
data.groupby(group_key).size()
data.groupby(group_key).count()
data.groupby(group_key).mean()

,0
East,-0.100594
West,0.960416


In [55]:
def fill_mean(group):
    return group.fillna(group.mean())

data.groupby(group_key).apply(fill_mean)

East  Ohio          0.329939
      New York      0.981994
      Vermont      -0.100594
      Florida      -1.613716
West  Oregon        1.561587
      Nevada        0.960416
      California    0.359244
      Idaho         0.960416
dtype: float64

In [56]:
fill_values = {"East": 0.5, "West": -1}
def fill_func(group):
    return group.fillna(fill_values[group.name])

data.groupby(group_key).apply(fill_func)

East  Ohio          0.329939
      New York      0.981994
      Vermont       0.500000
      Florida      -1.613716
West  Oregon        1.561587
      Nevada       -1.000000
      California    0.359244
      Idaho        -1.000000
dtype: float64

## Example: Random Sampling and Permutation

In [57]:
suits = ["H", "S", "C", "D"]  # Hearts, Spades, Clubs, Diamonds
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ["A"] + list(range(2, 11)) + ["J", "K", "Q"]
cards = []
for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

deck = pd.Series(card_val, index=cards)

In [58]:
deck.head(13)

,0
AH,1
2H,2
3H,3
4H,4
5H,5
6H,6
7H,7
8H,8
9H,9
10H,10


In [59]:
def draw(deck, n=5):
    return deck.sample(n)
draw(deck)

,0
4D,4
QH,10
8S,8
7D,7
9C,9


In [60]:
def get_suit(card):
    # last letter is suit
    return card[-1]

deck.groupby(get_suit).apply(draw, n=2)

C  6C     6
   KC    10
D  7D     7
   3D     3
H  7H     7
   9H     9
S  2S     2
   QS    10
dtype: int64

In [61]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

,0
AC,1
3C,3
5D,5
4D,4
10H,10
7H,7
QS,10
7S,7


##Example: Group Weighted Average and Correlation

In [62]:
df = pd.DataFrame({"category": ["a", "a", "a", "a",
                                "b", "b", "b", "b"],
                   "data": np.random.standard_normal(8),
                   "weights": np.random.uniform(size=8)})
df

,category,data,weights
0,a,-1.691656,0.955905
1,a,0.511622,0.012745
2,a,-0.401675,0.137009
3,a,0.968578,0.763037
4,b,-1.818215,0.492472
5,b,0.279963,0.832908
6,b,-0.200819,0.658331
7,b,-0.217221,0.612009


In [63]:
grouped = df.groupby("category")
def get_wavg(group):
    return np.average(group["data"], weights=group["weights"])

grouped.apply(get_wavg)

/tmp/ipython-input-505683550.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped.apply(get_wavg)


,0
category,
a,-0.495807
b,-0.357273


In [65]:
import pandas as pd
import numpy as np

# Tạo dữ liệu giả: 4 mã cổ phiếu trong 100 ngày
dates = pd.date_range("2009-01-01", periods=100)
np.random.seed(0)

data = {
    "AAPL": np.random.randint(100, 200, size=100),
    "MSFT": np.random.randint(20, 50, size=100),
    "XOM":  np.random.randint(50, 100, size=100),
    "SPX":  np.random.randint(1000, 2000, size=100)
}

close_px = pd.DataFrame(data, index=dates)
print(close_px.info())
print(close_px.tail(4))


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 100 entries, 2009-01-01 to 2009-04-10
Freq: D
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   AAPL    100 non-null    int64
 1   MSFT    100 non-null    int64
 2   XOM     100 non-null    int64
 3   SPX     100 non-null    int64
dtypes: int64(4)
memory usage: 3.9 KB
None
            AAPL  MSFT  XOM   SPX
2009-04-07   179    31   67  1284
2009-04-08   113    44   98  1276
2009-04-09   185    36   85  1978
2009-04-10   148    44   77  1324


In [66]:
def spx_corr(group):
    return group.corrwith(group["SPX"])

In [67]:
rets = close_px.pct_change().dropna()

In [69]:
def get_year(x):
    return x.year

by_year = rets.groupby(get_year)
by_year.apply(spx_corr)

,AAPL,MSFT,XOM,SPX
2009,0.074112,-0.008663,-0.089521,1.0


In [70]:
def corr_aapl_msft(group):
    return group["AAPL"].corr(group["MSFT"])
by_year.apply(corr_aapl_msft)

,0
2009,-0.148729


##Example: Group-Wise Linear Regression

In [71]:
import statsmodels.api as sm
def regress(data, yvar=None, xvars=None):
    Y = data[yvar]
    X = data[xvars]
    X["intercept"] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [72]:
by_year.apply(regress, yvar="AAPL", xvars=["SPX"])

,SPX,intercept
2009,0.074815,0.036496


## Group Transforms and “Unwrapped” GroupBys

In [73]:
df = pd.DataFrame({'key': ['a', 'b', 'c'] * 4,
                   'value': np.arange(12.)})
df

,key,value
0,a,0.0
1,b,1.0
2,c,2.0
3,a,3.0
4,b,4.0
5,c,5.0
6,a,6.0
7,b,7.0
8,c,8.0
9,a,9.0


In [74]:
g = df.groupby('key')['value']
g.mean()

,value
key,
a,4.5
b,5.5
c,6.5


In [75]:
def get_mean(group):
    return group.mean()
g.transform(get_mean)

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [76]:
g.transform('mean')

,value
0,4.5
1,5.5
2,6.5
3,4.5
4,5.5
5,6.5
6,4.5
7,5.5
8,6.5
9,4.5


In [77]:
def times_two(group):
    return group * 2
g.transform(times_two)

,value
0,0.0
1,2.0
2,4.0
3,6.0
4,8.0
5,10.0
6,12.0
7,14.0
8,16.0
9,18.0


In [78]:
def get_ranks(group):
    return group.rank(ascending=False)
g.transform(get_ranks)

,value
0,4.0
1,4.0
2,4.0
3,3.0
4,3.0
5,3.0
6,2.0
7,2.0
8,2.0
9,1.0


In [79]:
def normalize(x):
    return (x - x.mean()) / x.std()

In [80]:
g.transform(normalize)
g.apply(normalize)

key    
a    0    -1.161895
     3    -0.387298
     6     0.387298
     9     1.161895
b    1    -1.161895
     4    -0.387298
     7     0.387298
     10    1.161895
c    2    -1.161895
     5    -0.387298
     8     0.387298
     11    1.161895
Name: value, dtype: float64

In [81]:
g.transform('mean')
normalized = (df['value'] - g.transform('mean')) / g.transform('std')
normalized

,value
0,-1.161895
1,-1.161895
2,-1.161895
3,-0.387298
4,-0.387298
5,-0.387298
6,0.387298
7,0.387298
8,0.387298
9,1.161895


## Pivot Tables and Cross-Tabulation

In [82]:
tips.head()
tips.pivot_table(index=["day", "smoker"],
                 values=["size", "tip", "tip_pct", "total_bill"])

/tmp/ipython-input-3752735119.py:2: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  tips.pivot_table(index=["day", "smoker"],


size       tip   tip_pct  total_bill
day  smoker                                          
Thur Yes     2.352941  3.030000  0.163863   19.190588
     No      2.488889  2.673778  0.160298   17.113111
Fri  Yes     2.066667  2.714000  0.174783   16.813333
     No      2.250000  2.812500  0.151650   18.420000
Sat  Yes     2.476190  2.875476  0.147906   21.276667
     No      2.555556  3.102889  0.158048   19.661778
Sun  Yes     2.578947  3.516842  0.187250   24.120000
     No      2.929825  3.167895  0.160113   20.506667

In [83]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"])

/tmp/ipython-input-2917527286.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  tips.pivot_table(index=["time", "day"], columns="smoker",


size             tip_pct          
smoker            Yes        No       Yes        No
time   day                                         
Lunch  Thur  2.352941  2.500000  0.163863  0.160311
       Fri   1.833333  3.000000  0.188937  0.187735
Dinner Thur       NaN  2.000000       NaN  0.159744
       Fri   2.222222  2.000000  0.165347  0.139622
       Sat   2.476190  2.555556  0.147906  0.158048
       Sun   2.578947  2.929825  0.187250  0.160113

In [84]:
tips.pivot_table(index=["time", "day"], columns="smoker",
                 values=["tip_pct", "size"], margins=True)

/tmp/ipython-input-2226797760.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  tips.pivot_table(index=["time", "day"], columns="smoker",


size                       tip_pct                    
smoker            Yes        No       All       Yes        No       All
time   day                                                             
Lunch  Thur  2.352941  2.500000  2.459016  0.163863  0.160311  0.161301
       Fri   1.833333  3.000000  2.000000  0.188937  0.187735  0.188765
Dinner Thur       NaN  2.000000  2.000000       NaN  0.159744  0.159744
       Fri   2.222222  2.000000  2.166667  0.165347  0.139622  0.158916
       Sat   2.476190  2.555556  2.517241  0.147906  0.158048  0.153152
       Sun   2.578947  2.929825  2.842105  0.187250  0.160113  0.166897
All          2.408602  2.668874  2.569672  0.163196  0.159328  0.160803

In [85]:
tips.pivot_table(index=["time", "smoker"], columns="day",
                 values="tip_pct", aggfunc=len, margins=True)

/tmp/ipython-input-1240995350.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  tips.pivot_table(index=["time", "smoker"], columns="day",


day            Thur   Fri   Sat   Sun  All
time   smoker                             
Lunch  Yes     17.0   6.0   NaN   NaN   23
       No      44.0   1.0   NaN   NaN   45
Dinner Yes      NaN   9.0  42.0  19.0   70
       No       1.0   3.0  45.0  57.0  106
All            62.0  19.0  87.0  76.0  244

In [86]:
tips.pivot_table(index=["time", "size", "smoker"], columns="day",
                 values="tip_pct", fill_value=0)

/tmp/ipython-input-2877780592.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  tips.pivot_table(index=["time", "size", "smoker"], columns="day",


day                     Thur       Fri       Sat       Sun
time   size smoker                                        
Lunch  1    Yes     0.000000  0.223776  0.000000  0.000000
            No      0.181728  0.000000  0.000000  0.000000
       2    Yes     0.158843  0.181969  0.000000  0.000000
            No      0.166005  0.000000  0.000000  0.000000
       3    Yes     0.204952  0.000000  0.000000  0.000000
...                      ...       ...       ...       ...
Dinner 4    Yes     0.000000  0.117750  0.124515  0.193370
            No      0.000000  0.000000  0.150096  0.148143
       5    Yes     0.000000  0.000000  0.106572  0.065660
            No      0.000000  0.000000  0.000000  0.206928
       6    No      0.000000  0.000000  0.000000  0.103799

[21 rows x 4 columns]

##Cross-Tabulations: Crosstab

In [87]:
from io import StringIO
data = """Sample  Nationality  Handedness
1   USA  Right-handed
2   Japan    Left-handed
3   USA  Right-handed
4   Japan    Right-handed
5   Japan    Left-handed
6   Japan    Right-handed
7   USA  Right-handed
8   USA  Left-handed
9   Japan    Right-handed
10  USA  Right-handed"""
data = pd.read_table(StringIO(data), sep="\s+")

<>:13: SyntaxWarning: invalid escape sequence '\s'
<>:13: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipython-input-30910860.py:13: SyntaxWarning: invalid escape sequence '\s'
  data = pd.read_table(StringIO(data), sep="\s+")


In [88]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [89]:
pd.crosstab(data["Nationality"], data["Handedness"], margins=True)

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [90]:
pd.crosstab([tips["time"], tips["day"]], tips["smoker"], margins=True)

smoker       Yes   No  All
time   day                
Lunch  Thur   17   44   61
       Fri     6    1    7
Dinner Thur    0    1    1
       Fri     9    3   12
       Sat    42   45   87
       Sun    19   57   76
All           93  151  244

In [91]:
pd.options.display.max_rows = PREVIOUS_MAX_ROWS